In [ ]:
! pip install snntorch --quiet
! pip install tonic --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
# imports
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


# import tonic
# import tonic.transforms as transforms
# from torch.utils.data import DataLoader
# from torch.nn.utils.rnn import pad_sequence
# import librosa

#collatefn

import matplotlib.pyplot as plt
import numpy as np
import itertools

import tonic
import tonic.transforms as transforms
from tonic import collation
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch


In [ ]:
batch_size = 256

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

In [ ]:
# # Define the ResizeAudio transformation class
# class ResizeAudio(object):
#     def __init__(self, target_duration):
#         self.target_duration = target_duration

#     def __call__(self, audio):
#         # Calculate the current duration of the audio
#         current_duration = librosa.get_duration(filename=train_ds)

#         # Calculate the ratio to resize the audio
#         resize_ratio = self.target_duration / current_duration

#         # Resample the audio
#         audio_resized = audio.resample(resize_ratio)

#         return audio_resized

Downloading the data from tonic

In [ ]:
DATADIR = "/tmp/data"

sensor_size = tonic.datasets.SSC.sensor_size


dt = 12000  # All time units in Tonic are in microseconds (us)
encoding_dim = 100
target_duration = 10


# Define the transformation pipeline
# dense_transform = transforms.Compose([
#     transforms.Downsample(spatial_factor=encoding_dim / 700),
#     transforms.CropTime(max=1e6),
#     transforms.ToFrame(sensor_size=(encoding_dim, 1, 1), time_window=dt, include_incomplete=True),
#     ResizeAudio(target_duration)
# ])


dt = 12000  # all time units in Tonic in us
encoding_dim = 100

dense_transform = transforms.Compose(
    [
        transforms.Downsample(spatial_factor=encoding_dim / 700),
        transforms.CropTime(max=1e6),  # Adjust max time to control temporal size
        transforms.ToFrame(
            sensor_size=(encoding_dim, 1, 1), time_window=dt
        ),
    ]
)
train_ds = tonic.datasets.SSC(save_to=DATADIR, transform=dense_transform, split="train")
test_ds = tonic.datasets.SSC(save_to=DATADIR, transform=dense_transform, split='test')

In [ ]:
# train_loader = DataLoader(train_ds, batch_size, shuffle=True, drop_last=True)
# test_loader = DataLoader(test_ds, batch_size, shuffle=True, drop_last=True)
# def collate_fn(batch):
#     """A custom collate function to pad sequences to equal length."""
#     data, targets = zip(*batch)
#     data = pad_sequence(data, batch_first=True, padding_value=0)
#     targets = torch.tensor(targets)
#     return data, targets
# def collate_fn(batch):
#     """A custom collate function to pad sequences to equal length."""
#     data, targets = zip(*batch)
#     data = [torch.from_numpy(d) for d in data]  # Convert NumPy arrays to Tensors
#     data = pad_sequence(data, batch_first=True, padding_value=0)
#     targets = torch.tensor(targets)
#     return data, targets

# def collate_fn(batch):
#     """A custom collate function to pad sequences to equal length."""
#     data, targets = zip(*batch)
#     # Ensure data is a tensor and convert to float type
#     data = [torch.tensor(d).float() for d in data]
#     data = pad_sequence(data, batch_first=True, padding_value=0)
#     targets = torch.tensor(targets)
#     return data, targets

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


In [ ]:
# Network Architecture
num_inputs = 100
num_hidden = 1000
num_outputs = 35 #number ofwords thatARE there


# Temporal Dynamics
num_steps = 25
beta = 0.95
# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        self.lif2 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()

        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        for step in range(num_steps):
            cur1 = self.fc1(x)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)

# Load the network onto CUDA if available
net = Net().to(device)


Accuracy

In [ ]:
# pass data into the network, sum the spikes over time
# and compare the neuron with the highest number of spikes
# with the target

def print_batch_accuracy(data, targets, train=False):
    output, _ = net(data.view(batch_size, -1))
    _, idx = output.sum(dim=0).max(1)
    acc = np.mean((targets == idx).detach().cpu().numpy())

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

def train_printer():
    print(f"Epoch {epoch}, Iteration {iter_counter}")
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print_batch_accuracy(data, targets, train=True)
    print_batch_accuracy(test_data, test_targets, train=False)
    print("\n")

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))

In [ ]:
num_epochs = 1
loss_hist = []
test_loss_hist = []
counter = 0

# Outer training loop
for epoch in range(num_epochs):
    iter_counter = 0
    train_batch = iter(train_loader)

    # Minibatch training loop
    for data, targets in train_batch:
        data = data.to(device).float()
        targets = targets.to(device)

        # forward pass
        net.train()
        spk_rec, mem_rec = net(data.view(batch_size, -1))

        # initialize the loss & sum over time
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(num_steps):
            loss_val += loss(mem_rec[step], targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())

        # Test set
        with torch.no_grad():
            net.eval()
            test_data, test_targets = next(iter(test_loader))
            test_data = test_data.to(device).float()
            test_targets = test_targets.to(device)

            # Test set forward pass
            test_spk, test_mem = net(test_data.view(batch_size, -1))

            # Test set loss
            test_loss = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
                test_loss += loss(test_mem[step], test_targets)
            test_loss_hist.append(test_loss.item())

            # Print train/test loss/accuracy
            if counter % 50 == 0:
                train_printer()
            counter += 1
            iter_counter +=1

IndexError: index 670 is out of bounds for axis 2 with size 100

In [ ]:


# Assuming the Net class and other necessary configurations are defined as in your initial setup

# Dataset and DataLoader setup
DATADIR = "/tmp/data"
batch_size = 32
dt = 4000  # Time window for SSC dataset

# Define transformations
dense_transform = transforms.Compose([
    transforms.ToFrame(sensor_size=(encoding_dim, 1, 1), time_window=dt, include_incomplete=True),
    # Add any dataset-specific transforms here
])

# SSC dataset
train_ds = tonic.datasets.SSC(save_to=DATADIR, transform=dense_transform, split="train")
test_ds = tonic.datasets.SSC(save_to=DATADIR, transform=dense_transform, split="test")

def collate_fn(batch):
    """Custom collate function for padding sequences."""
    data, targets = zip(*batch)
    data = [torch.tensor(d, dtype=torch.float) for d in data]  # Ensure data is converted to tensors
    data = pad_sequence(data, batch_first=True, padding_value=0)
    targets = torch.tensor(targets, dtype=torch.long)  # Ensure targets are tensors
    return data, targets

# DataLoader
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = Net().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

def train_epoch(loader, model, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, targets) in enumerate(loader):
        data, targets = data.to(device), targets.to(device)

        # Forward pass
        output, _ = model(data)
        output = output.sum(dim=0)  # Sum over time dimension
        loss = loss_fn(output, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = output.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = correct / total
    return avg_loss, accuracy

def validate_epoch(loader, model, loss_fn, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in loader:
            data, targets = data.to(device), targets.to(device)

            # Forward pass
            output, _ = model(data)
            output = output.sum(dim=0)  # Sum over time dimension
            loss = loss_fn(output, targets)

            total_loss += loss.item()
            _, predicted = output.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = correct / total
    return avg_loss, accuracy

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(train_loader, net, loss_fn, optimizer, device)
    valid_loss, valid_acc = validate_epoch(test_loader, net, loss_fn, device)

    print(f"Epoch {epoch+1}:")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_acc:.4f}")


IndexError: index 189 is out of bounds for axis 2 with size 100